## Applied Data Science Project

Hugo Bossi, Fabian Frieden, Domenic Largiadèr

### Import the necessary libraries

In [1]:
# importing the libraries
from bs4 import BeautifulSoup #That is what we need for scrapping
import requests
import pandas as pd

In [2]:
#Learned in DS Fundamentals
import requests

# Request a websites content
page = requests.get("https://www.basketball-reference.com/leagues/NBA_2022_ratings.html")
#page = requests.get("https://www.nfl.com/standings/league/2021/REG").text

# Parsing the page content with BeautifulSoup
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
# soup = BeautifulSoup(html_content, "lxml") --> Alternative
# Print the page content
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
   <link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202205101" rel="dns-prefetch"/>
   <!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
   <script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://quantcast.mgr.consensu.org'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, '/choice.js')
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_N

In [3]:
#For more than one table, but not necessary for our case
print('Classes of each table:')
for table in soup.find_all('table'):
    print(table.get('class'))

Classes of each table:
['sortable', 'stats_table']


In [4]:
# Reduce the soup to the section of interest, namely our table.
table = soup.find("table")
#print(table)
columns = table.find("thead").find_all("th")
#print(columns)

colnames = []
for th in columns:
    colnames.append(th.text.strip())
#Removing unnecessary columnes
colnames.remove('Unadjusted')
colnames.remove('Adjusted')
colnames.remove('')
colnames.remove('Rk')
 

print(colnames)

['Team', 'Conf', 'Div', 'W', 'L', 'W/L%', 'MOV', 'ORtg', 'DRtg', 'NRtg', 'MOV/A', 'ORtg/A', 'DRtg/A', 'NRtg/A']


In [5]:
table_rows = table.find("tbody").find_all("tr")
#print(table_rows)
data = []
for tr in table_rows:
    td = tr.find_all("td")
    tdtext = [cell.get_text(strip = True) for cell in td]
    data.append(tdtext)

type(data)
data

[['Boston Celtics',
  'E',
  'A',
  '51',
  '31',
  '.622',
  '7.28',
  '115.37',
  '107.71',
  '7.66',
  '7.02',
  '115.05',
  '107.65',
  '7.39'],
 ['Phoenix Suns',
  'W',
  'P',
  '64',
  '18',
  '.780',
  '7.50',
  '115.65',
  '108.17',
  '7.48',
  '6.94',
  '115.41',
  '108.50',
  '6.91'],
 ['Utah Jazz',
  'W',
  'NW',
  '49',
  '33',
  '.598',
  '6.04',
  '117.68',
  '111.52',
  '6.16',
  '5.67',
  '117.63',
  '111.84',
  '5.79'],
 ['Golden State Warriors',
  'W',
  'P',
  '53',
  '29',
  '.646',
  '5.54',
  '113.33',
  '107.68',
  '5.65',
  '5.52',
  '113.24',
  '107.62',
  '5.62'],
 ['Memphis Grizzlies',
  'W',
  'SW',
  '56',
  '26',
  '.683',
  '5.68',
  '115.50',
  '109.94',
  '5.56',
  '5.37',
  '115.38',
  '110.14',
  '5.24'],
 ['Miami Heat',
  'E',
  'SE',
  '53',
  '29',
  '.646',
  '4.45',
  '114.83',
  '110.21',
  '4.62',
  '4.23',
  '114.61',
  '110.21',
  '4.40'],
 ['Dallas Mavericks',
  'W',
  'SW',
  '52',
  '30',
  '.634',
  '3.30',
  '113.82',
  '110.27',
  '3.55

In [6]:
df = pd.DataFrame(data, columns=colnames)

#df = df.set_index(["NFL Team"])
# Originally, I wanted to set the team name as Index, but as you can see this column will have to be cleaned first.
# As this will occur in the next step, I decided to leave the index as is and remove later on.

#df

In [7]:
#Adding the Ranking of the Teams manually
#Rk = ['1', '2', '3', '4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30'] 
#df['Rk'] = Rk

#Removing unnecessary columnes (we only go for adjusted values)
#del df['MOV']
#del df['ORtg']
#del df['DRtg']
#del df['NRtg']

df

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,Boston Celtics,E,A,51,31,.622,7.28,115.37,107.71,7.66,7.02,115.05,107.65,7.39
1,Phoenix Suns,W,P,64,18,.780,7.50,115.65,108.17,7.48,6.94,115.41,108.50,6.91
2,Utah Jazz,W,NW,49,33,.598,6.04,117.68,111.52,6.16,5.67,117.63,111.84,5.79
3,Golden State Warriors,W,P,53,29,.646,5.54,113.33,107.68,5.65,5.52,113.24,107.62,5.62
4,Memphis Grizzlies,W,SW,56,26,.683,5.68,115.50,109.94,5.56,5.37,115.38,110.14,5.24
5,Miami Heat,E,SE,53,29,.646,4.45,114.83,110.21,4.62,4.23,114.61,110.21,4.40
6,Dallas Mavericks,W,SW,52,30,.634,3.30,113.82,110.27,3.55,3.12,113.76,110.41,3.35
7,Milwaukee Bucks,E,C,51,31,.622,3.35,115.86,112.56,3.31,3.22,115.86,112.67,3.18
8,Philadelphia 76ers,E,A,51,31,.622,2.61,114.39,111.80,2.59,2.57,114.42,111.86,2.56
9,Toronto Raptors,E,A,48,34,.585,2.29,113.93,111.55,2.38,2.37,113.86,111.40,2.46


### Repeating for standing of other years

In [8]:
# Make a GET request to fetch the raw HTML content
html_content21 = requests.get("https://www.basketball-reference.com/leagues/NBA_2021_ratings.html")
html_content20 = requests.get("https://www.basketball-reference.com/leagues/NBA_2020_ratings.html")
html_content19 = requests.get("https://www.basketball-reference.com/leagues/NBA_2019_ratings.html")
html_content18 = requests.get("https://www.basketball-reference.com/leagues/NBA_2018_ratings.html")
html_content17 = requests.get("https://www.basketball-reference.com/leagues/NBA_2017_ratings.html")
html_content16 = requests.get("https://www.basketball-reference.com/leagues/NBA_2016_ratings.html")
html_content15 = requests.get("https://www.basketball-reference.com/leagues/NBA_2015_ratings.html")
html_content14 = requests.get("https://www.basketball-reference.com/leagues/NBA_2014_ratings.html")
html_content13 = requests.get("https://www.basketball-reference.com/leagues/NBA_2013_ratings.html")
html_content12 = requests.get("https://www.basketball-reference.com/leagues/NBA_2012_ratings.html")
html_content11 = requests.get("https://www.basketball-reference.com/leagues/NBA_2011_ratings.html")

# Parse the html content
soup21 = BeautifulSoup(html_content21.content, 'html.parser')
soup20 = BeautifulSoup(html_content20.content, 'html.parser')
soup19 = BeautifulSoup(html_content19.content, 'html.parser')
soup18 = BeautifulSoup(html_content18.content, 'html.parser')
soup17 = BeautifulSoup(html_content17.content, 'html.parser')
soup16 = BeautifulSoup(html_content16.content, 'html.parser')
soup15 = BeautifulSoup(html_content15.content, 'html.parser')
soup14 = BeautifulSoup(html_content14.content, 'html.parser')
soup13 = BeautifulSoup(html_content13.content, 'html.parser')
soup12 = BeautifulSoup(html_content12.content, 'html.parser')
soup11 = BeautifulSoup(html_content11.content, 'html.parser')

In [9]:
table21 = soup21.find("table")
table20 = soup20.find("table")
table19 = soup19.find("table")
table18 = soup18.find("table")
table17 = soup17.find("table")
table16 = soup16.find("table")
table15 = soup15.find("table")
table14 = soup14.find("table")
table13 = soup13.find("table")
table12 = soup12.find("table")
table11 = soup11.find("table")

In [10]:
table_rows21 = table21.find("tbody").find_all("tr")
table_rows20 = table20.find("tbody").find_all("tr")
table_rows19 = table19.find("tbody").find_all("tr")
table_rows18 = table18.find("tbody").find_all("tr")
table_rows17 = table17.find("tbody").find_all("tr")
table_rows16 = table16.find("tbody").find_all("tr")
table_rows15 = table15.find("tbody").find_all("tr")
table_rows14 = table14.find("tbody").find_all("tr")
table_rows13 = table13.find("tbody").find_all("tr")
table_rows12 = table12.find("tbody").find_all("tr")
table_rows11 = table11.find("tbody").find_all("tr")

#print(table_rows)
data21 = []
for tr in table_rows21:
    td = tr.find_all("td")
    tdtext = [cell.get_text(strip = True) for cell in td]
    data21.append(tdtext)

data20 = []
for tr in table_rows20:
    td = tr.find_all("td")
    tdtext = [cell.get_text(strip = True) for cell in td]
    data20.append(tdtext)

data19 = []
for tr in table_rows19:
    td = tr.find_all("td")
    tdtext = [cell.get_text(strip = True) for cell in td]
    data19.append(tdtext)

data18 = []
for tr in table_rows18:
    td = tr.find_all("td")
    tdtext = [cell.get_text(strip = True) for cell in td]
    data18.append(tdtext)

data17 = []
for tr in table_rows17:
    td = tr.find_all("td")
    tdtext = [cell.get_text(strip = True) for cell in td]
    data17.append(tdtext)

data16 = []
for tr in table_rows16:
    td = tr.find_all("td")
    tdtext = [cell.get_text(strip = True) for cell in td]
    data16.append(tdtext)

data15 = []
for tr in table_rows15:
    td = tr.find_all("td")
    tdtext = [cell.get_text(strip = True) for cell in td]
    data15.append(tdtext)

data14 = []
for tr in table_rows14:
    td = tr.find_all("td")
    tdtext = [cell.get_text(strip = True) for cell in td]
    data14.append(tdtext)

data13 = []
for tr in table_rows13:
    td = tr.find_all("td")
    tdtext = [cell.get_text(strip = True) for cell in td]
    data13.append(tdtext)

data12 = []
for tr in table_rows12:
    td = tr.find_all("td")
    tdtext = [cell.get_text(strip = True) for cell in td]
    data12.append(tdtext)

data11 = []
for tr in table_rows11:
    td = tr.find_all("td")
    tdtext = [cell.get_text(strip = True) for cell in td]
    data11.append(tdtext)

data11

[['Miami Heat',
  'E',
  'SE',
  '58',
  '24',
  '.707',
  '7.46',
  '112.62',
  '104.35',
  '8.27',
  '6.76',
  '112.43',
  '104.94',
  '7.49'],
 ['Chicago Bulls',
  'E',
  'C',
  '62',
  '20',
  '.756',
  '7.32',
  '109.30',
  '101.07',
  '8.23',
  '6.53',
  '109.02',
  '101.65',
  '7.37'],
 ['Los Angeles Lakers',
  'W',
  'P',
  '57',
  '25',
  '.695',
  '6.11',
  '111.61',
  '105.00',
  '6.61',
  '6.01',
  '111.40',
  '104.89',
  '6.51'],
 ['San Antonio Spurs',
  'W',
  'SW',
  '61',
  '21',
  '.744',
  '5.71',
  '112.31',
  '106.23',
  '6.07',
  '5.86',
  '112.22',
  '105.97',
  '6.25'],
 ['Orlando Magic',
  'E',
  'SE',
  '52',
  '30',
  '.634',
  '5.46',
  '108.78',
  '102.87',
  '5.91',
  '4.92',
  '108.63',
  '103.31',
  '5.32'],
 ['Boston Celtics',
  'E',
  'A',
  '56',
  '26',
  '.683',
  '5.37',
  '107.36',
  '101.46',
  '5.90',
  '4.83',
  '107.03',
  '101.73',
  '5.30'],
 ['Denver Nuggets',
  'W',
  'NW',
  '50',
  '32',
  '.610',
  '4.76',
  '112.84',
  '108.03',
  '4.82

In [1]:
df21 = pd.DataFrame(data21, columns=colnames)
df20 = pd.DataFrame(data20, columns=colnames)
df19 = pd.DataFrame(data19, columns=colnames)
df18 = pd.DataFrame(data18, columns=colnames)
df17 = pd.DataFrame(data17, columns=colnames)
df16 = pd.DataFrame(data16, columns=colnames)
df15 = pd.DataFrame(data15, columns=colnames)
df14 = pd.DataFrame(data14, columns=colnames)
df13 = pd.DataFrame(data13, columns=colnames)
df12 = pd.DataFrame(data12, columns=colnames)
df11 = pd.DataFrame(data11, columns=colnames)


# Test whether it worked on one of the new dataframes
df21

NameError: name 'pd' is not defined